In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
    
data=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data")
target=list(data.columns)[-1]
print(data)

      vhigh vhigh.1      2   2.1  small   low  unacc
0     vhigh   vhigh      2     2  small   med  unacc
1     vhigh   vhigh      2     2  small  high  unacc
2     vhigh   vhigh      2     2    med   low  unacc
3     vhigh   vhigh      2     2    med   med  unacc
4     vhigh   vhigh      2     2    med  high  unacc
...     ...     ...    ...   ...    ...   ...    ...
1722    low     low  5more  more    med   med   good
1723    low     low  5more  more    med  high  vgood
1724    low     low  5more  more    big   low  unacc
1725    low     low  5more  more    big   med   good
1726    low     low  5more  more    big  high  vgood

[1727 rows x 7 columns]


In [5]:
def train_test_split(dataset): 
    d=dataset.sample(frac=1)
    n=int(len(dataset)*0.75)
    
    
    training_data = d.iloc[:n].reset_index(drop=True)
    testing_data = d.iloc[n:].reset_index(drop=True)
    return(training_data,testing_data)

In [6]:
def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy


In [7]:
    
def InfoGain(data,split_attribute_name):
    total_entropy = entropy(data[target])
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculating the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain


In [8]:

def id3(dat,originaldata,features,parent_node_class = None):
    #checking for trivial conditions
    #1.
    if len(np.unique(dat[target]))<= 1:
        return np.unique(dat[target])[0]
    #2.
    #If the dataset is empty, return the mode target feature value in the original dataset
    elif len(dat)==0:
        return np.unique(originaldata[target])[np.argmax(np.unique(originaldata[target],return_counts=True)[1])]
    #3.
    #If the feature space is empty, return the mode target feature value of the direct parent node 
    elif len(features) ==0:
        return parent_node_class
    
    #non-trivial condition
    
    else:
        #Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = np.unique(dat[target])[np.argmax(np.unique(dat[target],return_counts=True)[1])]
        
        #Select the feature which best splits the dataset
        item_values = [InfoGain(dat,feature) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        tree = {best_feature:{}}
        
        
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]
        
        #Grow a branch under the root node for each possible value of the root node feature
        
        for value in np.unique(dat[best_feature]):
            value = value
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = dat.where(dat[best_feature] == value).dropna()
            
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
            subtree = id3(sub_data,data,features,parent_node_class)
            
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree
            
        return(tree)    


In [9]:
def predict(query,tree,default = 1):
    
    #1.
    for key in list(query.keys()):
        if key in list(tree.keys()):
            #2.
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            #3.
            result = tree[key][query[key]]
            #4.
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result


In [11]:
def accuracy(data,tree):
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    predicted=pd.DataFrame(columns=["predicted"])    
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    return((np.sum(predicted["predicted"] == data[target])/len(data))*100)
    

In [12]:
highest_accuracy=0.0
for i in range(10):
    training_data = train_test_split(data)[0]
    testing_data = train_test_split(data)[1] 
    tree = id3(training_data,training_data,training_data.columns[:-1]) 
    if accuracy(testing_data,tree)>highest_accuracy:
        highest_accuracy=accuracy(testing_data,tree)
        tree_final=tree
        
pprint(tree_final)                                                                   #display tree                                                                                         #evaluate accuracy
print('The prediction accuracy is: ',highest_accuracy,'%')
    

{'low': {'high': {'2.1': {'2': 'unacc',
                          '4': {'vhigh': {'high': {'vhigh.1': {'high': 'acc',
                                                               'low': 'acc',
                                                               'med': 'acc',
                                                               'vhigh': 'unacc'}},
                                          'low': {'vhigh.1': {'high': {'small': {'big': 'vgood',
                                                                                 'med': 'acc',
                                                                                 'small': 'acc'}},
                                                              'low': {'small': {'big': 'vgood',
                                                                                'med': {'2': {'2': 'good',
                                                                                              '4': 'vgood',
                                           